In [ ]:
%%html
<style>
  .container { width: 80%; }
  .float-img-left { float: left; }
  .float-img-right { float: right; }
</style>

In [ ]:
from IPython.display import Image

# Statistik

- Pius: Sieg / Niederlage gegen andere KIs & Stockfish auf verschiedenen Leveln
- Pius: Cache Hits / Misses / Tries d.h. Effektivität vom Cache (auch Vergleich mit / ohne Memoisierung)
- Philipp: Zugzeit von verschiedenen KIs und Einstellungen vergleichen
- Philipp: Entwicklung der KIs darstellen
- Anton: Belastung von Arbeitsspeicher in Zusammenhang mit der Tiefe und der Memoisierung (Cache Größe)
- Anton: SVE: Durchschnittliche Tiefe (bezogen auf die maximal Tiefe der Config)
- TBD: Vergleich zwischen Schwarz und Weiß als Farbe

Da sie im Folgenden oft referenziert werden wird hier noch einmal eine Übersicht über die verschiedenen KI-Versionen gegeben.

- `AI-Base-Class`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `Simplified Evaluation Function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: Die fünfte Version der AI, welche die dritte Version um das Alpha-Beta-Pruning erweitert.
- `Exercise06AI`: Die sechste Version der AI, welche die fünfte Version erneut um Memoisierung erweitert.
- `Exercise07AI`: Die siebte Version der AI, welche die sechste Version um Progressive Deepening erweitert.
- `Exercise08AI`: Die achte Version der AI, welche die siebte Version um die Singular Value Extension erweitert.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish Engine gesteuert wird.

## Ressourcenverbrauch der verschiedenen KI-Versionen

Die CPU-Auslastung ist bei jeder KI-Version maximal, d.h. es wird durchgängig die maximale verfügbare Rechenleistung genutzt. Die KI ist aktuell nicht parallelisiert, d.h. es wird jeweils nur ein CPU-Kern genutzt. Der Festplattenspeicher bewegt sich innerhalb von wenigen MB Verbrauch und ist somit vernachlässigbar. Die kritische Ressource ist der Arbeitsspeicher. Der genutzte Speicher unterscheidet sich je nach KI-Version und Konfiguration stark und ist (neben der Berechnungszeit) ein limitierender Faktor.

### Vergleich des RAM-Verbrauchs

Im Folgenden wird der Verbrauch des Arbeitsspeichers für die KI-Version 5 bis 8 verglichen. Die Versionen 1 bis 4 verfügen noch nicht über das Alpha-Beta-Pruning welches die Berechnungen stark beschleuningt, daher sind sie für einen Vergleich weniger interessant.

#### Messaufbau

Für jede gewählte Konfiguration und KI-Version wurden drei Testspiele gegen die Stockfish-KI gespielt. Die Spiele wurden ohne GUI auf auf einem Debian Linux gestartet. Der virtuelle Server wurde exklusiv für diesen Test erstellt und verfügt über 32 GB Arbeitsspeicher.  
Während den Spielen wurde der freie Arbeitsspeicher des Gesamtsystems im Abstand von 30 Sekunden gemessen. Der Verbrauch des Systems im Leerlauf wurde anschließend herausgerechnet. Die Messungenauigkeit wird auf etwa 15 MB geschätzt (maximale Abweichungen im Leerlauf).


#### Erläuterungen zur Tabelle
Für den RAM wurde jeweils das Maximum über alle drei Spiele ermittelt. Hierbei ist anzumerken, dass die Maximalwerte für ein einzelnes Spiel teilweise um mehr als 50 % vom Mittelwert der Maximalwerte aller Spiele abweicht, die Varianz ist hier also sehr hoch. Die Exercise05AI besitzt keine Memoisierung, somit gibt es keine Messung für den Cache. Bei der Exercise08AI wurd die maximale Tiefe für die Singular Value Extension jeweils um 1 höher gewählt als die normale Tiefe (minimaler Wert) um so weiterhin vergleichbar zu sein.

| Tiefe | Exercise05AI RAM | Exercise06AI RAM | Exercise07AI RAM | Exercise08AI RAM |
|-------|------------------|------------------|------------------|------------------|
| 3     | 128 MB           | 128 MB           | 305 MB           | 635 MB           |
| 4     | 138 MB           | 305 MB           | 658 MB           | 1010 MB          |
| 5     | 186 MB           | 1189 MB          | 3240 MB          | 2973 MB          |

Es zeigt sich, dass die Memoisierung den größten Einfluss auf den Arbeitsspeicher hat. Während der Verbrauch ohne Memoisierung nur leicht ansteigt (Exercise05AI), passiert dies deutlich stärker wenn Memoisierung eingesetzt wird (Exercise06AI). Bei der iterativen Suche (Exercise07AI) erhöht sich der Verbrauch abermals und bleibt bei der Exercise08AI in etwa gleich.

#### Exemplarischer Verlauf der Arbeitsspeicherbelastung
Das folgende Bild zeigt exemplarisch den Verlauf der RAM-Nutzung für ein Spiel der Exercise08AI gegen Stockfish. Gemessen wurde jeweils im Abstand von 30 Sekunden bei einer Suchtife von 4 und maximaler SVE-Tiefe von 8. Die vertikalen Striche markieren dabei das Ende eines Suchvorganges, d.h. einen abgeschlossenen Halbzug. Da Stockfish jeweils nur 0,1 Sekunden für den Zug benötigt kann dieser zeitliche Anteil hier vernachlässigt werden. Bei den steilen Anstiegen im Diagramm handelt es sich jeweils um eine Folge von Zügen mit vergleichsweise hoher durchschnittlicher Tiefe, d.h. es wurden viele Stellungen untersucht. Dort wo im Diagramm die Auslastung auf das Basisniveau von etwa 200 MB fällt wurde entsprechend Cache geleert nach einem irreversiblen Zug.

In [ ]:
Image(url="images/RAM_Ex08AI_vs_Stockfish.png", width=1200)

#### Arbeitsspeicherbelastung bei höherer Maximaltiefe

Die folgende Tabelle zeigt die Arbeitsspeicherbelastung für die Exercise08AI in verschiedenen Konfigurationen. Als Vergleichswert wurde noch die durchschnittliche Zeit für einen Halbzug und die durchschnittliche Suchtiefe hinzugefügt. Die maximale SVE-Tiefe ist die Tiefe bis zu der die Singular Value Extension durchgeführt wird. Ist diese Tiefe erreicht wird die Suche in diesem Teilbaum abgebrochen. Die durchschnittliche Tiefe wird als Summe der Suchtiefe bei jedem Blatt des Suchbaumes, geteilt durch die Anzahl der Summanden definiert. Ein Blatt ensteht sobald eine der Abbruchbedingungen für die Rekursion zutrifft.  
Es zeigt sich, dass die RAM-Auslastung bei höherer Maximaltiefe stark ansteigt und bei einer SVE-Tiefe von 12 die Suche auf den meisten aktuellen Rechnern bereits zu viel Arbeitsspeicher benötigt. Auch die Zeit pro Zug steigt stark. Hierbei ist zu beachten, dass die Zugzeit vorallem bei den ersten Zügen sehr hoch ist und bei einer SVE-Tiefe von 12 beispielsweise bis zu 52 Minuten für einzelne Züge möglich sind.

| Max. SVE Tiefe | Tiefe | RAM        | Durchschnittliche Berechnungszeit pro Zug | Durchschnittlich erreichte Suchtiefe |
|---------------|-------|------------|-------------------------------------------|--------------------------------------|
| 4             | 3     | 635 MB     | 0,75 s                                    | 3,49                                 |
| 5             | 4     | 1010 MB    | 10,29 s                                   | 4,27                                 |
| 6             | 5     | 2431 MB    | 51,44 s                                   | 5,10                                |
| 8             | 4     | 1756 MB    | 20,96 s                                   | 5,63                                 |
| 10            | 4     | 6695 MB    | 52,38 s                                   | 6,14                                 |
| 12            | 4     | > 31606 MB | 82,18 s                                   | 4,92                                 |

Die Daten zeigen insgesamt einerseits das die Memoisierung viel Arbeitsspeicher erfordert, aber dieser auch effektiv wieder freigegeben wird sobald der Cache durch einen irreversiblen Zug nutzlos wird. Die zweite Tabelle zeigt aber ebenso, dass der Arbeitsspeicher durchaus ein Limit für die Suche darstellen kann (wenn auch nur bei einer sehr langen Rechenzeit).

# Zugzeit der AIs

Bei den einzelnen Entwicklungsstufen der AI lassen sich klare zeitliche Unterschiede erkennen. Während die Memoisierung einen gewissen, aber leider nur geringen Geschwindigkeitsvorteil ermöglicht, bietet das Alpha-Beta-Pruning die größte Zeitersparnis.


Als `max_depth` Parameter wurde bei den folgenden Statistiken für die finale KI-Version jeweils der doppelte Wert der gegebenen Depth verwendet.
Wie in den nachfolgenden Abbildungen zu sehen ist, erhöhen sowohl die Einführung des Progressive Deepenings als auch der Singular Value Extension (SVE) den benötigten Zeitaufwand immens. Im Falle des Progressive Deepening wird der Zeitaufwand bei einer Tiefe von 4 um über 100 % erhöht, die SVE erhöht den Rechenaufwand sogar um mehr als den Faktor 13. Während die erhöhte Rechendauer der finalen KI-Version durch die Spielergebnisse gerechtfertigt werden kann, wie im nächsten Kapitel gezeigt wird, berechnet die `Exercise07AI` dieselben Züge wie ihre Vorgängerversion. Das Progressive Deepening sorgt für eine andere Sortierung der Züge, weshalb andere Ergebnisse möglich sind, aber das eigentliche Ziel einer Reduzierung der Berechnungsdauer wurde nicht erreicht.

Bei einer Depth von drei beträgt die durchschnittliche Zugzeit aller KIs unter eine Minute. Während die `Exercise03AI` mit 51,2 Sekunden die längste Rechendauer vorweist, ist die `Exercise06AI` mit 9,4 Sekunden die schnellste KI.

<img src="images/Depth3AvgDuration.png" width="700px" />

Die Berechnungsdauer wächst exponentiell mit steigender Suchtiefe. Auch bei einer Depth von 4 bleibt die sechste KI Version mit 72,9 Sekunden die schnellste KI. Mit 2117,9 Sekunden pro Zug ist `Exercise03AI` erneut mit Abstand die KI mit der längsten Berechnungsdauer. Selbst die finale Version der KI, die durch die Singular-Value-Extension bis zu acht Züge im Voraus berechnet, ist mit 1035,1 Sekunden mehr als doppelt so schnell.

<img src="images/Depth4AvgDuration.png" width="700px" />

### Exercise08AI

Durch die Verwendung der Singular Value Extension und dem dazugehörigen Parameter `max_depth` wird die finale Version der KI separat betrachtet und mit unterschiedlichen Werten für `max_depth` parametrisiert.

<img src="images/Depth3MaxDepthAvgDuration.png" width="700px" />
<img src="images/Depth4MaxDepthAvgDuration.png" width="700px" />

Wie zu sehen ist, fluktuieren die Ergebnisse bei einer Depth von drei noch relativ stark, während die durchschnittliche Zugzeit bei einer Depth von 4 konstant zunimmt und damit für ein erwartetes Ergebnis sorgt.



# Entwicklung der einzelnen KIs

Dieses Kapitel stellt die Entwicklung der einzelnen KI-Versionen dar, wobei immer eine KI mit ihrem direkten Nachfolger verglichen wird. Die ersten beiden KI-Iteration werden hierbei übergangen, da die erste KI nur zufällige Züge auswählt und somit für keine andere KI ein ernsthafter Gegner ist und auch die zweite Iteration der KI, welche den Minimax mit einfachem Materialwert zur Evaluierung verwendet, in den allermeisten Fällen höchstens auf ein Unentschieden hoffen kann.

Die dritte und vierte KI-Iteration, welche zur Evaluierung einer Stellung nun auf die Piece-Square Tables zurückgreifen, werden hier gemeinsam betrachtet, da die `Exercise04AI` die dritte Iteration lediglich um eine Memoisierung ergänzt, was zwar die Performanz steigert, aber nichts an der Stärke der KI ändert. Während diese beiden KI-Iterationen nie gegen die ersten beiden Versionen verlieren, so gewinnen sie auch niemals gegen spätere Versionen der KI. Auffallend ist, dass die dritte und vierte KI-Version im Spiel gegen sich selbst auf der schwarzen Seite in der Hälfte aller Fälle gewinnen und umgekehrt auf der weißen Seite zu 50 % verlieren. Die andere Hälfte der Spiele besteht hierbei jeweils zum Großteil aus Remis. Diese Eigenschaft ist kontraintuitiv, da im Schach eigentlich die weiße Seite als besser angesehen wird, was auch die Spielergebnisse der späteren KI-Versionen größtenteils wiedergespiegeln.

Auch die fünfte und sechste Version der KI werden gemeinsam betrachtet. Diese Versionen erweitern den Minimax um das Alpha-Beta-Pruning, wobei die sechste Iteration zusätzlich eine Memoisierung implementiert. Während das Alpha-Beta-Pruning grundsätzlich nichts an der Spielstärke verändert, hat es wie zuvor gezeigt einen starken Einfluss auf die Geschwindigkeit, mit der ein neuer Zug berechnet werden kann. Durch den gewonnenen Geschwindigkeitsbonus sind diese KI-Versionen in der Lage, mit einer höheren Depth als ihre Vorgängerversionen zu spielen und werden dadurch zu einem merklich stärkeren Gegner. Die folgenden Statistiken stellen einen Vergleich der vierten und sechsten Version im Spiel gegen die anderen KIs dar:

<div>
<img src="images/AI04Results.png" width="600px" />
<img src="images/AI06Results.png" width="600px" />
</div>

Wie bereits im vorherigen Kapitel beleuchtet, bringt das in der `Exercise07AI` implementierte Progressive Deepening keinen Vorteil in der Spielstärke der KI, sondern sollte lediglich die Performanz verbessern. Durch das Progressive Deepening findet eine andere Sortierung der Züge statt, wodurch sich bei gleicher Evaluierung mehrerer Züge der von den KI-Versionen ausgewählte Zug unterscheidet. Dass in dem Diagramm die Ergebnisse der fünften und sechsten KI-Version von denen der siebten abweichen, ist also als Zufall zu betrachten.

Die achte und damit finale Version KI hingegen bringt durch die Implementierung der Singular-Value-Extension einen signifikanten Vorteil. Wie in der folgenden Statistik zu sehen ist, gewinnt die KI nun ziemlich sicher gegen alle anderen KI Versionen. Im Spiel gegen sich selbst ist die KI jedoch analog zur dritten und vierten Version auf der schwarzen Seite stärker als auf der weißen.

<img src="images/AI08Results.png" width="600px" />


### Vergleich von Sieg und Niederlage

Für den Vergleich von Sieg und Niederlage wurden jeweils mindestens 10 Spiele gespielt. Die Diagramme zeigen die Gewinne in Prozenz der einzelnen KI-Versionen gegen Stockfish. Bei der Generierung der Statistiken wurde Stockfish auf eine ELO-Zahl von `1500` und eine Zug-Zeit von `0.1` Sekunden limitiert. Die ELO-Zahl von Stockfish wurde mit einer Bedenkzeit von `60s+0.6s` und in der `CCRL 40/4` festgesetzt. Aufgrund der Menge an benötigten Spielen wurde nur mit den Tiefen `3` und `4` gespielt. Die ersten zwei Diagramme zeigen die Ergebnisse mit Tiefe 3, die letzten zwei mit einer Tiefe von 4.

Alle KI-Versionen spielen mit einer Tiefe von 4 ungefähr doppelt so gut als mit einer Tiefe von 3. Schnell erkennbar ist in allen vier der folgenden Diagrammen, dass `Exercise01AI` durchweg verliert. Dies liegt an der Auswahl zuffälliger Züge im Mittelspiel. Die `Exercise02AI`, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt gewinnt mit einer Tiefe von 3 durchschnittlich 4 % der Spiele, mit einer Tiefe von 4 schon durchschnittlich 16 %. Hierbei hängt das Ergebnis stark von der Farbe ab, mit der die KI spielt. `Exercise03AI`, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `Simplified Evaluation Function` auswählt, wird vor allem weiß spielend ungefähr doppelt so gut. Die Beschleunigung durch die Memoisierung kommt bei `Exercise04AI` zu tragen, diese gewinnt auf einer Tiefe von 4 im Schnitt bereits die Hälfte der Spiele.

Die KI-Versionen `Exercise05AI`, `Exercise06AI` und `Exercise07AI` spielen verhältnismäßig sehr ähnlich. Dies ist darauf zurückzuführen, dass hierbei nicht das "Können" sondern durch das implementierte Alpha-Beta-Pruning, die erneute Memoisierung und das Progressive Deepening lediglich die Geschwindkeit optimiert wurde. 

Mit mehr als 54 % gewonnener Spiele auf einer Tiefe von 4 besiegt `Exercise08AI` Stockfish. Diese Verbesserung kann auf die implementierte Singular Value Extension zurückgeführt werden.

<img src="images/ResultsWhiteDepth3.png" width="600px" class="float-img-left" />
<img src="images/ResultsBlackDepth3.png" width="600px" class="float-img-right" />
<img src="images/ResultsWhiteDepth4.png" width="600px" class="float-img-left" />
<img src="images/ResultsBlackDepth4.png" width="600px" class="float-img-right" />

### Die Effektivität des Caches

Während jedes Spiels wurde pro Halbzug der Zugriff auf den implementierten Cache gemessen. Hiefür wurden die Variablen `cache_hits` und `cache_tries` für jede Farbe implementiert. Nach einem abgeschlossenen Spiel konnten so über alle `cache_hits` und `cache_tries` der jeweiligen Farbe die Summe gebildet werden und folglich mithilfe der Formel

$$ effectiveness = \frac{cache\_hits}{cache\_tries} $$

für ein gesamtes Spiel die Effektivität des Caches in Prozent je Farbe berechnet werden. Die zwei Diagramme unten zeigen den Mittelwert der Effektivität der jeweiligen KI-Versionen bei mehr als 50 Spielen.

<img src="images/CacheHitsWhite.png" width="600px" class="float-img-left" />
<img src="images/CacheHitsBlack.png" width="600px" class="float-img-right" />